In [1]:
# Import libraries
from flair.data import Sentence
from flair.models import SequenceTagger
from cassis import *

/home/allan/snap/jupyter/common/lib/python3.7/site-packages/joblib/_multiprocessing_helpers.py:45: UserWarning: [Errno 13] Permission denied.  joblib will operate in serial mode
  warnings.warn('%s.  joblib will operate in serial mode' % (e,))


In [2]:
tsfile = 'ts.xml'
csfile = './out.xml/document.txt.xmi'

TOKEN_TYPE = "de.tudarmstadt.ukp.dkpro.core.api.segmentation.type.Token"
SENTENCE_TYPE = "de.tudarmstadt.ukp.dkpro.core.api.segmentation.type.Sentence"

# Load Typesystem
with open(tsfile, 'rb') as f:
    typesystem = load_typesystem(f)

# Load Cas
with open(csfile, 'rb') as f:
    cas = load_cas_from_xmi(f, typesystem=typesystem)

In [3]:
# Get annotation type
AnnotationType = typesystem.get_type(TOKEN_TYPE)

In [4]:
tokens = list(cas.select(TOKEN_TYPE))
words = [cas.get_covered_text(token) for token in tokens]
tokens

/snap/jupyter/6/lib/python3.7/site-packages/ipykernel_launcher.py:2: DeprecatedWarning: get_covered_text is deprecated. Use annotation.get_covered_text()
  


[de_tudarmstadt_ukp_dkpro_core_api_segmentation_type_Token(xmiID=4, parent=None, lemma=None, stem=None, pos=de_tudarmstadt_ukp_dkpro_core_api_lexmorph_type_pos_POS_PROPN(xmiID=85, begin=0, end=8, PosValue='NNP', coarseValue='PROPN', type='de.tudarmstadt.ukp.dkpro.core.api.lexmorph.type.pos.POS_PROPN'), morph=None, id=None, form=None, syntacticFunction=None, begin=0, end=8, type='de.tudarmstadt.ukp.dkpro.core.api.segmentation.type.Token'),
 de_tudarmstadt_ukp_dkpro_core_api_segmentation_type_Token(xmiID=5, parent=None, lemma=None, stem=None, pos=de_tudarmstadt_ukp_dkpro_core_api_lexmorph_type_pos_POS_X(xmiID=86, begin=9, end=11, PosValue='FW', coarseValue='X', type='de.tudarmstadt.ukp.dkpro.core.api.lexmorph.type.pos.POS_X'), morph=None, id=None, form=None, syntacticFunction=None, begin=9, end=11, type='de.tudarmstadt.ukp.dkpro.core.api.segmentation.type.Token'),
 de_tudarmstadt_ukp_dkpro_core_api_segmentation_type_Token(xmiID=6, parent=None, lemma=None, stem=None, pos=de_tudarmstadt_uk

In [5]:
sentence = ''
for word in words:
    sentence += word + ' '

sentence = Sentence(sentence)
model_name = 'ner'
nlp = SequenceTagger.load(model_name)

nlp.predict(sentence)

2020-06-15 09:38:28,005 loading file /home/allan/snap/jupyter/6/.flair/models/en-ner-conll03-v0.4.pt


[Sentence: "Courtney de Souza has been living with her daughter Susan in Berlin for about 6 years ."   [− Tokens: 17  − Token-Labels: "Courtney <B-PER> de <I-PER> Souza <E-PER> has been living with her daughter Susan <S-PER> in Berlin <S-LOC> for about 6 years ."]]

### #testing start

In [6]:
NERType = typesystem.create_type("webanno.custom.flair-ner")
typesystem.add_feature(type_=NERType, name='ner', rangeTypeName='uima.cas.String')

In [12]:
begin = 0
for sent in range(len(sentence)):
    token = sentence[sent]
    word = str(token).split()
    tag = sentence[sent].get_tag('ner')
    end = begin + len(word)
    cas_sentence = AnnotationType(begin = tokens[sent].begin, 
                                  end = tokens[sent].end)
    ner_annotation = NERType(begin = tokens[sent].begin,
                      end = tokens[sent].end,
                      ner = tag.value)
    cas.add_annotation(cas_sentence)
    cas.add_annotation(ner_annotation)
    begin = end + 1
xmi = cas.to_xmi('./newCas.xmi', pretty_print=True)

### #testing end